SWE546 Final Project - Method 1

In this method, SGD is used and two matrices (A and B) is obtained so that their product gives the minimum error with the ratings we know. 

Movielens 10M data set is used. The data set is divided into 2 part, one to be used for training, the other to be used for testing. 

The execution time for finding A and B was too long so only 10 iteration is performed. The result matrices are stored for future use to analyse performance metrics. 

After the A and B is found, the test data is used to calculate accuracy, precision, recall, TPR and FPR. These metrics are calculated by selecting a threshold of 4 as rating and the expectation was if the rating in the test data is >= than the threshold, then the prediction must also be >= than the threshold and the other way around. Any prediction that does not follow this metric is counted as either false positive or false negative. 

The performance metrics can be found as printed at end of this notebook.



In [2]:
import numpy as np
import pandas as pd
import time


rnames = ['user_id', 'movie_id', 'rating', 'timestamp'] 
mnames = ['MovieID', 'Title', 'Genres']
ratings = pd.read_table('../data/ml-10M100K/ratings.dat', sep='::',engine='python', header=None, names=rnames)
train = pd.read_table('../data/ml-10M100K/ra.train', sep='::',engine='python', header=None, names=rnames)
test = pd.read_table('../data/ml-10M100K/ra.test', sep='::',engine='python', header=None, names=rnames)
movies = pd.read_table('../data/ml-10M100K/movies.dat', sep='::',engine='python', header=None, names=mnames)

In [4]:
ratings

,user_id,movie_id,rating,timestamp
0,1,122,5,838985046
1,1,185,5,838983525
2,1,231,5,838983392
3,1,292,5,838983421
4,1,316,5,838983392
5,1,329,5,838983392
6,1,355,5,838984474
7,1,356,5,838983653
8,1,362,5,838984885
9,1,364,5,838983707


In [5]:
train

,user_id,movie_id,rating,timestamp
0,1,370,5.0,838984596
1,1,377,5.0,838983834
2,1,420,5.0,838983834
3,1,466,5.0,838984679
4,1,480,5.0,838983653
5,1,520,5.0,838984679
6,1,539,5.0,838984068
7,1,586,5.0,838984068
8,1,588,5.0,838983339
9,1,589,5.0,838983778


In [6]:
test

,user_id,movie_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392
5,1,329,5.0,838983392
6,1,355,5.0,838984474
7,1,356,5.0,838983653
8,1,362,5.0,838984885
9,1,364,5.0,838983707


In [3]:
Ys = np.mat(train)
Ys

matrix([[  1.00000000e+00,   3.70000000e+02,   5.00000000e+00,
           8.38984596e+08],
        [  1.00000000e+00,   3.77000000e+02,   5.00000000e+00,
           8.38983834e+08],
        [  1.00000000e+00,   4.20000000e+02,   5.00000000e+00,
           8.38983834e+08],
        ..., 
        [  7.15670000e+04,   2.29400000e+03,   5.00000000e+00,
           9.12577968e+08],
        [  7.15670000e+04,   2.33800000e+03,   2.00000000e+00,
           9.12578016e+08],
        [  7.15670000e+04,   2.38400000e+03,   2.00000000e+00,
           9.12578173e+08]])

In [4]:
Yt = np.mat(test)
Yt

matrix([[  1.00000000e+00,   1.22000000e+02,   5.00000000e+00,
           8.38985046e+08],
        [  1.00000000e+00,   1.85000000e+02,   5.00000000e+00,
           8.38983525e+08],
        [  1.00000000e+00,   2.31000000e+02,   5.00000000e+00,
           8.38983392e+08],
        ..., 
        [  7.15670000e+04,   4.80000000e+02,   4.00000000e+00,
           9.12580688e+08],
        [  7.15670000e+04,   5.89000000e+02,   4.00000000e+00,
           9.12580647e+08],
        [  7.15670000e+04,   7.80000000e+02,   4.00000000e+00,
           9.12580753e+08]])

In [5]:
Ym = np.mat(movies)

Nr = 71567 # Number of users
Nc = 10681 # Number of movies

Nid = 65133 # Max movie index

#MovieIDs are from 1 to 65133 but there is ratings for only 10681 movies in the data set.
#There is a mapping between the MovieID and index is needed. 
MovieMap = np.mat(np.zeros((Nid, 1)))

size = Ym.shape[0]

for i in range(size):
    MovieMap[Ym[i, 0]-1, 0] = i

MovieMap

matrix([[  0.00000000e+00],
        [  1.00000000e+00],
        [  2.00000000e+00],
        ..., 
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  1.06800000e+04]])

In [20]:
Nk = 2

#Start with random A and B
A = np.mat(np.random.rand(Nr, Nk))
B = np.mat(np.random.rand(Nk, Nc))

#Number of iterations are kept at 10 because of the execution time
EPOCH = 10
Eta = 0.05
eta = Eta

start_time = time.clock()

Ysize = Ys.shape[0]

for i in range(EPOCH):    
    for k in range(Ysize):
        u = Ys[k,0] - 1
        m = Ys[k,1] - 1
        m_idx = MovieMap[m, 0]
        err = Ys[k,2] - A[u,:]*B[:,m_idx]
        
        temp_A = A[u,:] + eta*err[0,0]*B[:,m_idx].T
        B[:,m_idx]   = B[:,m_idx] + eta*err[0,0]*A[u,:].T
        A[u,:]   = temp_A
    
    eta = Eta*1./(i+1)

time_sgd = time.clock() - start_time
print time_sgd, "seconds"

#Save the A and B matrices for future use - no need to run this cell again
A.dump("matrixA.dat")
B.dump("matrixB.dat")



6916.45 seconds


/home/yesim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/yesim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [24]:
A = np.load("matrixA.dat")
A

matrix([[ 1.65280637,  1.88101146],
        [ 0.76248237,  1.20261728],
        [ 0.95834816,  1.64353172],
        ..., 
        [ 1.27394563,  1.3091596 ],
        [ 1.52035809,  1.23438469],
        [ 1.438594  ,  0.95890416]])

In [25]:
B = np.load("matrixB.dat")
B

matrix([[ 0.23770542,  0.80324166,  0.76156334, ...,  1.12340907,
          0.7149995 ,  1.44162324],
        [ 0.65595517,  0.12163754,  0.06734896, ...,  1.54459447,
          0.68618707,  1.5625731 ]])

In [23]:
testSize = Yt.shape[0]

np.random.shuffle(Yt)

#The threshold is chosen to be 4. Every rating above 4 will be counted as "good" and every rating below 
#will be caounted as "bad" rating. 
metric_threshold = 4


TP = 0;
FP = 0; 
TN = 0;
FN = 0;

for t in range(testSize):
    user = Yt[t, 0] - 1
    movie = Yt[t, 1] - 1
    m_index = MovieMap[movie, 0]
    
    rating = Yt[t, 2]
    
    #Find out the prediction for the user, movie pair in the test data (A*B)
    predict = A[user,:]*B[:,m_index]
    
    if (rating >= metric_threshold and predict >= metric_threshold):
        TP += 1
    elif (rating < metric_threshold and predict < metric_threshold):
        TN += 1
    elif (rating > metric_threshold and predict < metric_threshold):
        FN += 1
    elif (rating < metric_threshold and predict > metric_threshold):
        FP += 1
    
if ((TP + FN) == 0 ):
    TPR = 0
else:
    TPR = float(TP / float(TP+FN))
if ((FP + TN) == 0 ):
    FPR = 0
else:
    FPR = float(FP / float(FP+TN))
        
print('truePos = ' + repr(TP) + ', trueNeg = ' + repr(TN) + 
      ', falsePos = ' + repr(FP) + ', falseNeg = ' + repr(FN) + '\n')

print('True Positive Rate = ' + repr(TPR) + ', False Positive Rate = ' + repr(FPR) + '\n')

accuracy = float((TP + TN) / float(TP + FP + TN + FN)) * 100
precision = float(TP / float(TP + FP)) * 100
recall = float(TP / float(TP + FN)) * 100

print('accuracy = ' + repr(accuracy) + '%, precision = ' + repr(precision) + 
      '%, recall = ' + repr(recall) + '%\n')


truePos = 158666, trueNeg = 275355, falsePos = 35189, falseNeg = 86059

True Positive Rate = 0.6483440596588007, False Positive Rate = 0.11331405533515379

accuracy = 78.16409704125388%, precision = 81.84777281989115%, recall = 64.83440596588007%

